In [82]:
import pandas as pd
import openpyxl
import json
import csv
import requests
from datetime import datetime


In [ ]:
#Q1

# Load JSON data from the URL
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)
data = response.json()

# Load country code excel file 
country_df = pd.read_excel('./Country-Code.xlsx')

# Extract required fields
restaurant_data = []

# print(country_df)
# print(data[0]['restaurants'][0]['restaurant']['R']['res_id'])
# print(data[0]['restaurants'][0]['restaurant']['name'])
# print(data[0]['restaurants'][0]['restaurant']['location']['country_id'])
# print(data[0]['restaurants'][0]['restaurant']['location']['city'])
# print(data[0]['restaurants'][0]['restaurant']['user_rating']['votes'])
# print(data[0]['restaurants'][0]['restaurant']['user_rating']['aggregate_rating'])
# print(data[0]['restaurants'][0]['restaurant']['cuisines'])

for restaurants in data:
    for restaurant in restaurants['restaurants']:
        res_id = restaurant['restaurant']['R']['res_id']
        res_name = restaurant['restaurant']['name']
        country_id = restaurant['restaurant']['location']['country_id']
        city = restaurant['restaurant']['location']['city']
        votes = restaurant['restaurant']['user_rating']['votes']
        agg_rating = restaurant['restaurant']['user_rating']['aggregate_rating']
        cuisines = restaurant['restaurant']['cuisines']
        
        # Append row to the DataFrame with each restaurant details
        restaurant_data.append({'res_id': res_id, 'res_name': res_name, 'Country Code': country_id,
                                'city': city, 'votes': votes, 'agg_rating': agg_rating, 'cuisines': cuisines})
df = pd.DataFrame(restaurant_data)

# Merge the dataframes by country code
merged_df = pd.merge(df, country_df, how='inner', on='Country Code')
# Drop country code column
merged_df = merged_df.drop(columns=['Country Code'])

# df.to_csv('restaurants.csv', index=False)

In [ ]:
#Q2

# Load JSON data from the URL
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)
data = response.json()

# Extract required fields
event_data = []

for restaurants in data:
    for restaurant in restaurants['restaurants']:
        if 'zomato_events' in restaurant['restaurant']:
            for event in restaurant['restaurant']['zomato_events']:
                event_id = event['event']['event_id']
                res_id = restaurant['restaurant']['R']['res_id']
                res_name = restaurant['restaurant']['name']
                if len(event['event']['photos']) == 0:
                    photo_url = 'NA'
                else:
                    photo_url = event['event']['photos'][0]['photo']['url']
                event_title = event['event']['title']
                event_start = event['event']['start_date']
                event_end = event['event']['end_date']

                # Convert date from string to date
                event_start = datetime.strptime(event_start, "%Y-%m-%d")
                event_end = datetime.strptime(event_end, "%Y-%m-%d")

                # Check if the event is within 2019 April
                is_within_april = any(event_start <= datetime(2019, 4, day) <= event_end for day in range(1, 31))

                # Append row to the DataFrame with each event details
                if is_within_april:
                    event_data.append({'event_id': event_id, 'res_id': res_id, 'res_name': res_name, 'photo_url': photo_url,
                                       'event_title': event_title, 'event_start_date': event_start, 'event_end_date': event_end})
                    
df = pd.DataFrame(event_data)
# df.to_csv('restaurant_events.csv', index=False)



In [83]:
#Q3

# Load JSON data from the URL
url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"
response = requests.get(url)
data = response.json()

# Get data for user rating
restaurant_data = []
for restaurants in data:
    for restaurant in restaurants['restaurants']:
        rating_text = restaurant['restaurant']['user_rating']['rating_text']
        agg_rating = restaurant['restaurant']['user_rating']['aggregate_rating']
        # Append row to the DataFrame with each restaurant details
        restaurant_data.append({'rating_text': rating_text, 'agg_rating': agg_rating})

df = pd.DataFrame(restaurant_data)

rating_category = ['Excellent', 'Very Good', 'Good', 'Average', 'Poor']
threshold_list = []

# Get threshold for each rating category
for category in rating_category:
    ratings = df[df['rating_text'] == category]
    # Find the minimum and maximum of aggregate rating for each category
    min_agg_rating = ratings['agg_rating'].min()
    max_agg_rating = ratings['agg_rating'].max()

    # Append row to the DataFrame with each rating threshold
    threshold_list.append({'rating_text': category, 'min_threshold_agg_rating': min_agg_rating, 'max_threshold_agg_rating': max_agg_rating})

result_df = pd.DataFrame(threshold_list)
print(result_df)

  rating_text min_threshold_agg_rating max_threshold_agg_rating
0   Excellent                      4.5                      4.9
1   Very Good                      4.0                      4.4
2        Good                      3.5                      3.9
3     Average                      2.5                      3.4
4        Poor                      2.2                      2.2
